# Hugging Faceを使用した多言語推論器のファインチューニング

著者: [Edward Beeching](https://huggingface.co/edbeeching), [Quentin Gallouédec](https://huggingface.co/qgallouedec), [Lewis Tunstall](https://huggingface.co/lewtun)

[OpenAI o3](https://openai.com/index/introducing-o3-and-o4-mini/)のような大規模推論モデルは、応答の精度と品質を向上させるために思考の連鎖を生成します。しかし、これらのモデルの多くは、他の言語で質問されても英語で推論を行います。

このノートブックでは、OpenAIのオープンウェイト推論モデル[OpenAI gpt-oss-20b](https://huggingface.co/openai/gpt-oss-20b)を複数の言語で効果的に推論できるようにファインチューニングする方法を示します。モデルのシステムプロンプトに新しい_「推論言語」_オプションを追加し、多言語推論データセットでHugging Faceの[TRLライブラリ](https://github.com/huggingface/trl)を使用して[教師ありファインチューニング](https://huggingface.co/learn/llm-course/chapter11/1)を適用することでこれを実現します。

以下のステップを説明します：

1. **セットアップ:** 必要なライブラリをインストールします。
2. **データセットの準備:** ファインチューニング用のデータセットをダウンロードし、フォーマットします。
3. **モデルの準備:** ベースモデルを読み込み、メモリ効率的な技術である[LoRA](https://huggingface.co/learn/llm-course/chapter11/4)でファインチューニング用に設定します。
4. **ファインチューニング:** 多言語推論データでモデルを訓練します。
5. **推論:** ファインチューニングされたモデルを使用して、異なる言語で推論応答を生成します。

最終的な結果は、英語、スペイン語、フランス語、イタリア語、またはドイツ語で思考の連鎖を生成できる多言語推論モデルです。_言語を混在_させることも可能です。例えば、スペイン語で質問し、ドイツ語での推論を要求し、最終的な応答をスペイン語で受け取ることができます：

```txt
User:
    ¿Cuál es el capital de Australia?
Assistant reasoning:
    Okay, der Benutzer fragt nach der Hauptstadt Australiens. Ich erinnere mich, dass Canberra die Hauptstadt ist. Ich
    sollte das bestätigen. Lass mich sehen, ob es irgendwelche potenziellen Verwirrungen gibt. Der Benutzer könnte auch
    an der größten Stadt interessiert sein. Die größte Stadt ist Sydney, aber die Hauptstadt ist Canberra. Ich sollte
    das klarstellen. Vielleicht auch erwähnen, dass Canberra eine geplante Stadt ist und nicht die größte. Der Benutzer
    könnte auch nach der Geografie fragen. Vielleicht erwähne ich, dass Canberra im südwestlichen Teil der Australian
    Capital Territory liegt. Ich sollte die Antwort präzise und freundlich halten. Vielleicht auch erwähnen, dass
    Canberra oft mit Sydney verwechselt wird. Ich sollte sicherstellen, dass die Antwort klar und korrekt ist.
Assistant response:
    La capital de Australia es **Canberra**. Aunque es la ciudad más pequeña de las principales capitales del país, fue
    elegida en 1908 como la sede del gobierno federal para equilibrar la influencia entre las ciudades de Sydney y
    Melbourne. Canberra está ubicada en el Territorio de la Capital Australiana (ACT), en el este de Australia.
```

このチュートリアルが、十分に代表されていない言語で作業するAI開発者が、母国語での[`openai/gpt-oss-20b`](https://huggingface.co/openai/gpt-oss-20b)の解釈可能性を向上させることを可能にすることを願っています。

> **注意:** このノートブックは、80GBメモリを搭載した単一のH100 GPUで実行するように設計されています。より小さなGPUにアクセスできる場合は、以下のハイパーパラメータでバッチサイズとシーケンス長を減らすことができます。

## セットアップ

始めるために、必要なライブラリをすべてインストールしましょう。まずPyTorchをインストールします：

In [ ]:
%pip install torch --index-url https://download.pytorch.org/whl/cu128

次に、残りの依存関係をインストールします：

In [ ]:
%pip install "trl>=0.20.0" "peft>=0.17.0" "transformers>=4.55.0" trackio

最後に、以下のようにHugging Faceアカウントにログインします：

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

必要なライブラリをインストールしたので、ファインチューニングに使用するデータセットを見てみましょう。

## データセットの準備

[Multilingual-Thinking](https://huggingface.co/datasets/HuggingFaceH4/Multilingual-Thinking)を使用します。これは、思考の連鎖がフランス語、スペイン語、ドイツ語などの複数の言語に翻訳された推論データセットです。このデータセットで`openai/gpt-oss-20b`をファインチューニングすることで、これらの言語で推論ステップを生成することを学習し、それらの言語を話すユーザーが推論プロセスを理解できるようになります。

```html
<iframe
  src="https://huggingface.co/datasets/HuggingFaceH4/Multilingual-Thinking/embed/viewer/default/train"
  frameborder="0"
  width="100%"
  height="560px"
></iframe>
```

Hugging Face Hubからこのデータセットをダウンロードしましょう：

In [ ]:
from datasets import load_dataset

dataset = load_dataset("HuggingFaceH4/Multilingual-Thinking", split="train")
dataset

これは1,000例の小さなデータセットですが、広範囲にわたるポストトレーニングを受けた`openai/gpt-oss-20b`のようなモデルには通常これで十分です。トレーニング例の一つを見てみましょう：

In [ ]:
dataset[0]

`gpt-oss`モデルは、会話構造の定義、推論出力の生成、関数呼び出しの構造化のためのHarmony応答形式で訓練されました。この形式はOpenAI Responses APIを模倣するように設計されており、以下の表はデータセットで使用される異なるメッセージタイプをまとめています：

|||
| :---- | :--|
| `developer` | developer メッセージは、モデルに対するカスタム指示を提供するために使用されます（通常 `system` ロールと呼ばれるもの）。 |
| `user` | user メッセージは、モデルへの入力を提供するために使用されます。 |
| `assistant` | モデルによって出力され、ツール呼び出しまたはメッセージ出力のいずれかになります。出力は、メッセージの意図を識別する特定の「チャンネル」に関連付けられる場合もあります。 |
| `analysis` | これらは、モデルが思考の連鎖に使用しているメッセージです。 |
| `final` | final チャンネルでタグ付けされたメッセージは、エンドユーザーに表示することを意図したメッセージで、モデルからの応答を表します。 |
| `messages` | 上記の内容を組み合わせて完全な会話を作成するメッセージのリストです。これがモデルへの入力となります。 |

[OpenAI のメッセージ形式](https://platform.openai.com/docs/api-reference/messages/object)に馴染みがある方は、これが非常に似ていることがわかるでしょうが、重要な違いがあります：

> `assistant` ターンには2つの特別なフィールドが含まれています：モデルの推論プロセスを含む `thinking` フィールドと、ユーザーへの最終的な応答を含む `content` フィールドです。

モデルをファインチューニングするために、これらのメッセージをモデルが理解できる形式に変換する必要があります。実際には、各メッセージをモデルの[_チャットテンプレート_](https://huggingface.co/docs/transformers/chat_templating)でフォーマットし、その結果のテキストをトークン化することで行われます。TRLライブラリはこれを自動的に行いますが、どのように動作するかを理解するために、ステップバイステップで見ていきましょう。

そのために、まずトークナイザーを読み込みましょう：

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("openai/gpt-oss-20b")

その後、tokenizerの`apply_chat_template()`メソッドを使用してメッセージをフォーマットできます：

In [ ]:
messages = dataset[0]["messages"]
conversation = tokenizer.apply_chat_template(messages, tokenize=False)
print(conversation)

このチャットテンプレートは非常に洗練されているので、詳しく見てみましょう！まず、各メッセージの開始と終了を示す特別なトークン`<|start|>`と`<|end|>`があることがわかります。また、会話の終了を示す`<|return|>`トークンもあります。これらのトークンは、モデルが会話の構造を理解するのに役立ちます。

また、**2つ**のタイプのシステムメッセージがあることもわかります：

* すべてのメッセージに使用されるデフォルトの`system`メッセージ。上記の例では、これは_「You are ChatGPT, a large language model trained by OpenAI...」_というテキストを指します
* カスタム指示を含む特別な`developer`メッセージ（`messages`オブジェクトの`system`ロールで定義される）。これにより、特定の会話でモデルがどのように振る舞うべきかについて、追加のコンテキストを提供できます。上記の例では、これは_「You are an AI chatbot with a lively and energetic personality.」_というテキストを指します

最後に、アシスタントの応答が一連の**チャンネル**に含まれていることがわかります：

* `analysis`チャンネルは、モデルの推論プロセスに使用され、ユーザーの質問について段階的に考えることができます。上記の例では、これはフランス語のテキスト_「D'accord, l'utilisateur demande les tendances Twitter...」_を指します
* `final`チャンネルは、ユーザーに対するモデルの最終的な応答に使用されます。上記の例では、これは_「Hey there! While I can't check Twitter...」_というテキストを指します

データセットの準備方法を理解したので、次にトレーニング用のモデルの準備に進みましょう。

## モデルの準備

トレーニング用にモデルを準備するため、まず[Hugging Face Hub](https://huggingface.co)から重みをダウンロードしましょう。🤗 Transformersの`AutoModelForCausalLM`クラスを使用してモデルを読み込みます：

In [ ]:
import torch
from transformers import AutoModelForCausalLM, Mxfp4Config

quantization_config = Mxfp4Config(dequantize=True)
model_kwargs = dict(
    attn_implementation="eager",
    torch_dtype=torch.bfloat16,
    quantization_config=quantization_config,
    use_cache=False,
    device_map="auto",
)

model = AutoModelForCausalLM.from_pretrained("openai/gpt-oss-20b", **model_kwargs)

これにより、トレーニングに必要な設定でモデルが読み込まれます。`attn_implementation`はパフォーマンス向上のために`eager`に設定され、`use_cache`は勾配チェックポイントを使用してモデルをファインチューニングするため`False`に設定されています。

🤗 Transformersに慣れている方は、量子化に`Mxfp4Config`を使用していることにお気づきかもしれません。これはOpenAIモデル専用の設定で、AIワークロードに最適化された[MXFP4](https://en.wikipedia.org/wiki/Block_floating_point)と呼ばれる特別な4ビット浮動小数点形式を使用した混合精度トレーニングを可能にします。

モデルをトレーニングする前に、デフォルト設定でのモデルの動作を確認するためにサンプル応答を生成してみましょう。そのためには、サンプルプロンプトをトークン化し、モデルを使用して応答を生成する必要があります：

In [ ]:
messages = [
    {"role": "user", "content": "¿Cuál es el capital de Australia?"},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
).to(model.device)

output_ids = model.generate(input_ids, max_new_tokens=512)
response = tokenizer.batch_decode(output_ids)[0]
print(response)

この例では、モデルが最初に英語で質問について推論し、その後スペイン語で最終的な回答を提供していることがわかります。これはモデルのデフォルトの動作ですが、少しファインチューニングを行うことで変更できるかどうか見てみましょう。

そのために、[LoRA](https://huggingface.co/learn/llm-course/chapter11/4)（Low-Rank Adaptation）と呼ばれる技術を使用してモデルをファインチューニングします。この技術により、モデルの特定の層をいくつか調整することができ、`openai/gpt-oss-20b`のような大規模なモデルに特に有用です。

まず、モデルを`PeftModel`としてラップし、LoRA設定を定義する必要があります。これを行うために、[PEFTライブラリ](https://github.com/huggingface/peft)の`LoraConfig`クラスを使用します：

In [ ]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules="all-linear",
    target_parameters=[
        "7.mlp.experts.gate_up_proj",
        "7.mlp.experts.down_proj",
        "15.mlp.experts.gate_up_proj",
        "15.mlp.experts.down_proj",
        "23.mlp.experts.gate_up_proj",
        "23.mlp.experts.down_proj",
    ],
)
peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

ここでは、LoRAの基本的なハイパーパラメータを使用しましたが、異なる値を試して、それらがモデルの性能にどのような影響を与えるかを確認することができます。例えば、`r`を増やすと、より多くの訓練可能なパラメータが有効になり、より多くのVRAMと訓練時間が必要になる代わりに、より良いモデルが得られる可能性があります。

**注意：** `openai/gpt-oss-20b`モデルは[Mixture-of-Experts (MoE)](https://huggingface.co/blog/moe)アーキテクチャです。アテンション層をターゲットにする（`target_modules="all-linear"`）ことに加えて、エキスパートモジュール内のプロジェクション層も含めることが重要です。PEFTは`target_parameters`引数を通じてこれを促進し、`mlp.experts.down_proj`や`mlp.experts.gate_up_proj`などのエキスパート固有の層を指定することができます。この例では、これらのプロジェクション層のサブセットをターゲットにしていますが、異なる構成で実験することをお勧めします。

モデルとデータセットの準備ができたので、トレーニング用のハイパーパラメータを定義できます。

## ファインチューニング

TRLは、`SFTConfig`クラスを使用してトレーニング用のハイパーパラメータを定義する便利な方法を提供しています。学習率、バッチサイズ、エポック数、その他のパラメータを以下のように設定します：

In [ ]:
from trl import SFTConfig

training_args = SFTConfig(
    learning_rate=2e-4,
    gradient_checkpointing=True,
    num_train_epochs=1,
    logging_steps=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    max_length=2048,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine_with_min_lr",
    lr_scheduler_kwargs={"min_lr_rate": 0.1},
    output_dir="gpt-oss-20b-multilingual-reasoner",
    report_to="trackio",
    push_to_hub=True,
)

`per_device_train_batch_size`が4に設定され、`gradient_accumulation_steps`が4に設定されていることに注意してください。これは、1つのGPU上で実質的に4 x 4 = 16のバッチサイズを持つことを意味します。ハードウェア構成に基づいて、これらの値を調整する必要がある場合があります。また、トレーニングの進捗とメトリクスをログに記録するために[Trackio](https://huggingface.co/blog/trackio)を使用していますが、お好みの他のロギングライブラリを使用することもできます。

これで、モデルを訓練するために必要なすべての要素が揃いました。TRLの`SFTTrainer`クラスを使用して訓練プロセスを処理します。このトレーナーは、データセットのフォーマット、チャットテンプレートの適用、およびモデルの訓練を処理します：

In [ ]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=peft_model,
    args=training_args,
    train_dataset=dataset,
    processing_class=tokenizer,
)
trainer.train()

H100 GPUでは、これは約18分で訓練が完了しますが、お使いのハードウェアによってはより長い時間がかかる場合があります。

## モデルを保存してHugging Face Hubにプッシュする

最後に、ファインチューニングしたモデルをHubリポジトリにプッシュして、コミュニティと共有することができます：

In [ ]:
trainer.save_model(training_args.output_dir)
trainer.push_to_hub(dataset_name="HuggingFaceH4/Multilingual-Thinking")

**注意**: メモリ不足（OOM）エラーを避けるため、この時点でカーネルを再起動することをお勧めします。訓練済みモデルはまだGPUメモリを占有していますが、もはや必要ありません。

## 推論

モデルがHubにアップロードされたら、推論に使用できます。そのためには、まず元のベースモデルとそのトークナイザーを初期化します。次に、高速推論のために、ファインチューニングされた重みをベースモデルとマージする必要があります：

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("openai/gpt-oss-20b")

# Load the original model first
model_kwargs = dict(attn_implementation="eager", torch_dtype="auto", use_cache=True, device_map="auto")
base_model = AutoModelForCausalLM.from_pretrained("openai/gpt-oss-20b", **model_kwargs).cuda()

# Merge fine-tuned weights with the base model
peft_model_id = "gpt-oss-20b-multilingual-reasoner"
model = PeftModel.from_pretrained(base_model, peft_model_id)
model = model.merge_and_unload()

モデルが読み込まれたので、最後のステップはモデルからトークンを生成することです！ここでは、モデルの`generate`メソッドを使用して、入力プロンプトに基づいて出力を生成します。まず、プロンプトを定義しましょう：

これで、プロンプトをトークン化して出力を生成できます。最後に、出力トークンをデコードして最終的なレスポンスを取得できます：

In [ ]:
REASONING_LANGUAGE = "German"
SYSTEM_PROMPT = f"reasoning language: {REASONING_LANGUAGE}"
USER_PROMPT = "¿Cuál es el capital de Australia?"  # Spanish for "What is the capital of Australia?"

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": USER_PROMPT},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
).to(model.device)

gen_kwargs = {"max_new_tokens": 512, "do_sample": True, "temperature": 0.6, "top_p": None, "top_k": None}

output_ids = model.generate(input_ids, **gen_kwargs)
response = tokenizer.batch_decode(output_ids)[0]
print(response)

モデルが明示的にファインチューニングされていない言語、例えば中国語やヒンディー語でも試してみましょう：

In [ ]:
REASONING_LANGUAGE = "Chinese"  # or Hindi, or any other language...
SYSTEM_PROMPT = f"reasoning language: {REASONING_LANGUAGE}"
USER_PROMPT = "What is the national symbol of Canada?"

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": USER_PROMPT},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
).to(model.device)

output_ids = model.generate(input_ids, **gen_kwargs)
response = tokenizer.batch_decode(output_ids)[0]
print(response)

素晴らしい、うまく動作しました - `openai/gpt-oss-20b`を複数の言語で推論できるようにファインチューニングすることができました！

## 結論

おめでとうございます！TRLライブラリとLoRAを使用して、多言語推論モデルのファインチューニングに成功しました。このノートブックの手順は、Hugging Face Hub上の他の多くの[データセット](https://huggingface.co/datasets)で[`openai/gpt-oss-20b`](https://huggingface.co/openai/gpt-oss-20b)をファインチューニングするために応用することができます。あなたが何を構築するかを楽しみにしています！